# **DESAFIO 01 - EVASÃO BANCÁRIA (CHURN)**

Vamos usar [este conjunto de dados](https://github.com/cassiusf/datasets/raw/refs/heads/main/desafios/Churn_Dataset.csv) para tentar prever evasão de clientes no setor bancário.

Este conjunto de dados contém informações sobre clientes bancários que deixaram o banco ou continuam sendo seus clientes. O conjunto de dados inclui as seguintes variáveis:

- **Customer ID**: identificador único para cada cliente
- **Surname**: sobrenome do cliente
- **Credit Score**: valor numérico representando o score de crédito do cliente, ou seja, uma classificação interna do quão bom pagador ele é
- **Geography**: país onde o cliente reside (França, Espanha ou Alemanha)
- **Gender**: gênero do cliente (masculino ou feminino)
- **Age**: idade do cliente.
- **Tenure**: número de anos que o cliente está no banco.
- **Balance**: saldo da conta do cliente.
- **NumOfProducts**: número de produtos bancários utilizados pelo cliente (por exemplo, conta poupança, cartão de crédito).
- **HasCrCard**: se o cliente tem um cartão de crédito (1 = sim, 0 = não).
- **IsActiveMember**: se o cliente é um membro ativo (1 = sim, 0 = não).
- **EstimatedSalary**: renda estimada do cliente.
- **Exited**: evasão do cliente (1 = sim, 0 = não) ***(Variável alvo)***.


## **TAREFA**

Seu objetivo é utilizar um modelo de machine learning rasa para prever a possibilidade de um cliente cancelar sua conta no banco.

Você deverá fazer todas es etapas necessárias tanto na avaliação dos dados disponíveis quanto na preparação necessária para execução do modelo escolhido.

Todas as etapas devem ser documentadas e as escolhas justificadas.

A performance do seu modelo deverá ser avaliada pelas métricas Acurácia, Precision, Recall e F1-Score. Inclua em sua avaliação final de performance Curvas, ROC e/ou Precision-Recall, de acordo com a necessidade.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# Importa o conjunto de dados
df = pd.read_csv("https://raw.githubusercontent.com/cassiusf/datasets/refs/heads/main/desafios/Churn_Dataset.csv")
pd.reset_option('all')




In [ ]:
pd.set_option('display.max_columns', None)
df

In [ ]:
##verifiquei quantos nulos exitem

df.isna().sum()



In [ ]:
##decide excluir os nulos

df= df.dropna()




In [ ]:
##removendo colunas irrelevantes 

from sklearn.preprocessing import OneHotEncoder, StandardScaler

df.drop(columns=['CustomerId', 'Surname'], inplace=True)

In [ ]:
##codificando variveis categoricas 
df = pd.get_dummies(df,columns=["Geography","Gender"], drop_first=True)


In [ ]:
##normalização variaveis numericas 
scaler = StandardScaler()
num_cols = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
##criei uma tabela so de number para verificar  outlier
df_numerico = df.select_dtypes(include=['int64', 'float64'])
df_numerico = df_numerico.drop(columns=(['Exited',"IsActiveMember","HasCrCard","RowNumber",]))
df_numerico



In [ ]:
##fazendo boxplot para verificar os outlier de cada coluna
for coluna in df_numerico.columns:
    plt.figure(figsize=(5, 3))
    plt.boxplot(df_numerico[coluna])
    plt.title(f'Boxplot de {coluna}')
    plt.ylabel('Valores')
    plt.show()
    

In [ ]:
##sepração de treino e teste 

from sklearn.model_selection import train_test_split
##tirando variavel alvo
y = df["Exited"]
X = df.drop(columns=["Exited"])

#separando teste e treino 

X_train , X_test, y_train, y_test =  train_test_split(X,y,test_size=0.3)


In [20]:
from imblearn.over_sampling import SMOTE

##aplicando smote apenas nos dados de treino para 

smote = SMOTE(sampling_strategy=0.5)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train,y_train)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
#rodando modelo random forest 

modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

#fazendo predicao 

y_pred = modelo.predict(X_test)

In [ ]:
##verificando primeiras predicoes 
print("Primeiras previsões:", y_pred[:10])
print("Valores reais:", y_test[:10].values)  


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Criar a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# grafico para visualizar matriz de confusao 
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("matriz de confusao")
plt.show()